In [58]:
import os, io
import pandas as pd
from splunksdkexamples.search import main as search

def get_df_melt(d):
    argv=[
        d["spl"],
        '--output_mode=csv',
    ]
    print(f'processing scope: {d["scope"]}')
    results=search(argv)
    try:
        df = pd.read_csv(
            io.StringIO(
            results.read().decode('utf-8')
            ),
            encoding='utf8',
            sep=',',
            low_memory=False
        )
    except Exception:
        print(f'something went wrong with scope: {d["scope"]}')
        return None
    else:
        df1 = df.melt(id_vars=['_time', 'host', 'obj'])
        return df1.assign(scope=[d["scope"],] * len(df1.index))

In [59]:
span='1h'

spl_list = [
    {
        "scope" : "system",
        "spl": f"""
        search index=_introspection  sourcetype=splunk_resource_usage component=Hostwide
        | bin _time span={span}
        | rename data.* as * 
        | rename component as obj
        |eval cpu=cpu_system_pct+cpu_user_pct 
        | stats limit=0 avg(cpu) as avg_cpu%, avg(mem_used) as avg_mem_MiB by _time host obj
        """,
    },
    
    {
        "scope" : "proc",
        "spl" : f"""
        search index=_introspection  sourcetype=splunk_resource_usage component=PerProcess
        | rename data.* as * 
        | rename process as obj
        | eval pct_cpu = pct_cpu/6/60/24
        | eval mem_used = mem_used/6/60/24
        | bin _time span={span}
        | stats limit=0 sum(pct_cpu) as avg_cpu%, sum(mem_used) as avg_mem_MiB by _time host obj
        """,
    },

    {
        "scope": "proc_class",
        "spl" : f"""
        search index=_introspection sourcetype=splunk_resource_usage component=PerProcess 
        (host=sh1 OR host=idx1 OR host=master1)
        | eval process = 'data.process' | eval args = 'data.args' | eval pid = 'data.pid' 
        | eval ppid = 'data.ppid' | eval elapsed = 'data.elapsed' | eval mem_used = 'data.mem_used' 
        | eval mem = 'data.mem' | eval pct_memory = 'data.pct_memory' | eval pct_cpu = 'data.pct_cpu' 
        | eval fd = 'data.fd_used' 
        | eval t_count = 'data.t_count' 
        | eval sid = 'data.search_props.sid' | eval app = 'data.search_props.app' 
        | eval label = 'data.search_props.label' | eval type = 'data.search_props.type' 
        | eval mode = 'data.search_props.mode' | eval user = 'data.search_props.user' 
        | eval role = 'data.search_props.role' 
        | eval label = if(isnotnull('data.search_props.label'), 'data.search_props.label', "") 
        | eval provenance = if(isnotnull('data.search_props.provenance'), 'data.search_props.provenance', "unknown") 
        | eval search_head = case(isnotnull('data.search_props.search_head') 
        AND 'data.search_props.role' == "peer", 'data.search_props.search_head', isnull('data.search_props.search_head') 
        AND 'data.search_props.role' == "head", "_self", isnull('data.search_props.search_head') 
        AND 'data.search_props.role' == "peer", "_unknown") 
        | eval workload_pool = if(isnotnull('data.workload_pool'), 'data.workload_pool', "UNDEFINED") 
        | eval process_class = case( process=="splunk-optimize","index service", process=="sh" 
        OR process=="ksh" OR process=="bash" OR like(process,"python%") 
        OR process=="powershell","scripted input", process=="mongod", "KVStore") 
        | eval process_class = case( process=="splunkd" 
        AND (like(args,"-p %start%") OR like(args,"service") 
        OR like(args,"%_internal_launch_under_systemd%")),"splunkd server", process=="splunkd" 
        AND isnotnull(sid) AND type=="datamodel acceleration","datamodel acceleration", process=="splunkd" 
        AND isnotnull(sid) AND type=="scheduled","scheduler searches", process=="splunkd" 
        AND isnotnull(sid) AND type=="ad-hoc","ad-hoc searches", process=="splunkd" 
        AND isnotnull(sid) AND type=="summary index","summary index", process=="splunkd" 
        AND (like(args,"fsck%") OR like(args,"recover-metadata%") 
        OR like(args,"cluster_thing")),"index service", process=="splunkd" 
        AND args=="instrument-resource-usage", "scripted input", (like(process,"python%") 
        AND like(args,"%/appserver/mrsparkle/root.py%")) 
        OR like(process,"splunkweb"),"Splunk Web", isnotnull(process_class), process_class) 
        | eval process_class = if(isnull(process_class),"other",process_class) 
        | bin _time span=10s 
        | stats 
        latest(pct_cpu) AS resource_usage_dedup 
        latest(mem_used) AS mem_usage_dedup 
        latest(fd) AS fd_usage_dedup
        latest(t_count) AS t_count_usage_dedup
        latest(process_class) AS process_class by pid, _time, host 
        | stats sum(resource_usage_dedup) AS resource_usage 
        sum(mem_usage_dedup) AS mem_usage 
        sum(fd_usage_dedup) AS fd_usage 
        sum(t_count_usage_dedup) AS t_usage  
        by _time, process_class, host 
        | bin _time span={span} 
        | rename process_class as obj
        | stats 
        avg(resource_usage) AS "avg_cpu%" 
        avg(mem_usage) AS "avg_mem_MiB" 
        avg(fd_usage) AS "avg_fd_usage"
        avg(t_usage) AS "avg_threads"
        by _time, host, obj
        """,
    },
    {
        "scope" : "crash",
        "spl" : f"""
        search index=_internal source=*crash* 
        | rex field=_raw "Crashing thread: (?<crash_thread>\w+)" 
        | bin _time span={span} 
        | rename crash_thread as obj
        | stats count by _time host obj
        """,
        
    },
    {
        "scope" : "dma_fail",
        "spl" :     f"""
        search index=_internal sourcetype=scheduler 
        source=*/scheduler*.log search_type=datamodel_acceleration savedsearch_name=* status!=*_remote* 
        |rename savedsearch_name as obj
        |bin _time span={span}
        | stats distinct_count(scheduled_time) as total_count, 
        count(eval(status="success")) as success_count by _time host obj 
        | eval failed_count=total_count-success_count 
        | eval failed_ratio=failed_count/total_count*100
        """,
    },
    {
        "scope" : "buckets_tx",
        "spl" :     f"""
        search index="_audit" (action=remote_bucket_download OR action=local_bucket_upload 
        OR action=local_bucket_evict OR action=remote_bucket_remove) info=completed cache_id="*bid|*" 
        | rename action as obj
        |bin _time span={span}
        | stats sum(kb) as throughput_KB by _time host obj
        """,
    }
]

df_all = pd.concat(
    map(get_df_melt, spl_list)
)

processing scope: system
processing scope: proc
processing scope: proc_class
processing scope: crash
something went wrong with scope: crash
processing scope: dma_fail
processing scope: buckets_tx


In [60]:
df_all

,_time,host,obj,variable,value,scope
0,2022-12-29T14:00:00.000+00:00,deployer1,Hostwide,avg_cpu%,5.625759e+01,system
1,2022-12-29T14:00:00.000+00:00,hf1,Hostwide,avg_cpu%,1.542472e+01,system
2,2022-12-29T14:00:00.000+00:00,idx1,Hostwide,avg_cpu%,5.037933e+00,system
3,2022-12-29T14:00:00.000+00:00,idx2,Hostwide,avg_cpu%,5.167821e+00,system
4,2022-12-29T14:00:00.000+00:00,idx3,Hostwide,avg_cpu%,5.068315e+00,system
...,...,...,...,...,...,...
49995,2022-05-29T10:00:00.000+00:00,idx2,local_bucket_upload,throughput_KB,1.266412e+06,buckets_tx
49996,2022-05-29T10:00:00.000+00:00,idx2,remote_bucket_download,throughput_KB,0.000000e+00,buckets_tx
49997,2022-05-29T10:00:00.000+00:00,idx20,local_bucket_upload,throughput_KB,1.161016e+06,buckets_tx
49998,2022-05-29T10:00:00.000+00:00,idx20,remote_bucket_download,throughput_KB,3.066800e+04,buckets_tx


In [48]:
df_all.loc[(df_all['host']=='sh1') &  (df_all['obj']=='splunkd')]


,_time,host,obj,variable,value
29,2022-12-29T14:00:00.000+00:00,sh1,splunkd,avg_cpu%,0.813799
69,2022-12-29T15:00:00.000+00:00,sh1,splunkd,avg_cpu%,3.691743
107,2022-12-29T16:00:00.000+00:00,sh1,splunkd,avg_cpu%,1.911983
148,2022-12-29T17:00:00.000+00:00,sh1,splunkd,avg_cpu%,2.155941
188,2022-12-29T18:00:00.000+00:00,sh1,splunkd,avg_cpu%,2.184726
...,...,...,...,...,...
97324,2023-01-09T00:00:00.000+00:00,sh1,splunkd,avg_mem_MiB,124.328754
97891,2023-01-09T01:00:00.000+00:00,sh1,splunkd,avg_mem_MiB,124.488346
98456,2023-01-09T02:00:00.000+00:00,sh1,splunkd,avg_mem_MiB,126.200425
99027,2023-01-09T03:00:00.000+00:00,sh1,splunkd,avg_mem_MiB,133.464543


results
